<a href="https://colab.research.google.com/github/BraulioRivTri/Pruebas/blob/main/Proyecto1AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
from sklearn.metrics import r2_score, mean_squared_error
from scipy.stats import kstest
import statsmodels.regression.linear_model as rg
import statsmodels.stats.diagnostic as dg
from statsmodels.stats.stattools import durbin_watson
from statsmodels.formula.api import ols
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.stats.diagnostic import het_white
from math import log
from sklearn.feature_selection import f_regression

In [24]:
insurance=pd.read_csv(r'/content/insurance.csv',delimiter=',')
insurance

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520
...,...,...,...,...,...,...,...
1333,50,male,30.970,3,no,northwest,10600.54830
1334,18,female,31.920,0,no,northeast,2205.98080
1335,18,female,36.850,0,no,southeast,1629.83350
1336,21,female,25.800,0,no,southwest,2007.94500


In [25]:
#Información de mi base de datos
insurance.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1338 non-null   int64  
 1   sex       1338 non-null   object 
 2   bmi       1338 non-null   float64
 3   children  1338 non-null   int64  
 4   smoker    1338 non-null   object 
 5   region    1338 non-null   object 
 6   charges   1338 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 73.3+ KB


In [26]:
#Revisión de espacios vacíos
insurance.isna().sum()

age         0
sex         0
bmi         0
children    0
smoker      0
region      0
charges     0
dtype: int64

In [27]:
insurance=pd.get_dummies(insurance,columns=['sex'])
insurance

,age,bmi,children,smoker,region,charges,sex_female,sex_male
0,19,27.900,0,yes,southwest,16884.92400,1,0
1,18,33.770,1,no,southeast,1725.55230,0,1
2,28,33.000,3,no,southeast,4449.46200,0,1
3,33,22.705,0,no,northwest,21984.47061,0,1
4,32,28.880,0,no,northwest,3866.85520,0,1
...,...,...,...,...,...,...,...,...
1333,50,30.970,3,no,northwest,10600.54830,0,1
1334,18,31.920,0,no,northeast,2205.98080,1,0
1335,18,36.850,0,no,southeast,1629.83350,1,0
1336,21,25.800,0,no,southwest,2007.94500,1,0


In [28]:
insurance=pd.get_dummies(insurance,columns=['smoker'])
insurance


,age,bmi,children,region,charges,sex_female,sex_male,smoker_no,smoker_yes
0,19,27.900,0,southwest,16884.92400,1,0,0,1
1,18,33.770,1,southeast,1725.55230,0,1,1,0
2,28,33.000,3,southeast,4449.46200,0,1,1,0
3,33,22.705,0,northwest,21984.47061,0,1,1,0
4,32,28.880,0,northwest,3866.85520,0,1,1,0
...,...,...,...,...,...,...,...,...,...
1333,50,30.970,3,northwest,10600.54830,0,1,1,0
1334,18,31.920,0,northeast,2205.98080,1,0,1,0
1335,18,36.850,0,southeast,1629.83350,1,0,1,0
1336,21,25.800,0,southwest,2007.94500,1,0,1,0


In [29]:
insurance=pd.get_dummies(insurance,columns=['region'])
insurance

,age,bmi,children,charges,sex_female,sex_male,smoker_no,smoker_yes,region_northeast,region_northwest,region_southeast,region_southwest
0,19,27.900,0,16884.92400,1,0,0,1,0,0,0,1
1,18,33.770,1,1725.55230,0,1,1,0,0,0,1,0
2,28,33.000,3,4449.46200,0,1,1,0,0,0,1,0
3,33,22.705,0,21984.47061,0,1,1,0,0,1,0,0
4,32,28.880,0,3866.85520,0,1,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
1333,50,30.970,3,10600.54830,0,1,1,0,0,1,0,0
1334,18,31.920,0,2205.98080,1,0,1,0,1,0,0,0
1335,18,36.850,0,1629.83350,1,0,1,0,0,0,1,0
1336,21,25.800,0,2007.94500,1,0,1,0,0,0,0,1


In [30]:
data=insurance
data

,age,bmi,children,charges,sex_female,sex_male,smoker_no,smoker_yes,region_northeast,region_northwest,region_southeast,region_southwest
0,19,27.900,0,16884.92400,1,0,0,1,0,0,0,1
1,18,33.770,1,1725.55230,0,1,1,0,0,0,1,0
2,28,33.000,3,4449.46200,0,1,1,0,0,0,1,0
3,33,22.705,0,21984.47061,0,1,1,0,0,1,0,0
4,32,28.880,0,3866.85520,0,1,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
1333,50,30.970,3,10600.54830,0,1,1,0,0,1,0,0
1334,18,31.920,0,2205.98080,1,0,1,0,1,0,0,0
1335,18,36.850,0,1629.83350,1,0,1,0,0,0,1,0
1336,21,25.800,0,2007.94500,1,0,1,0,0,0,0,1


In [31]:
data['log_age']=np.log(data['age'])
data['log_bmi']=np.log(data['bmi'])
data['log_charges']=np.log(data['charges'])
data

,age,bmi,children,charges,sex_female,sex_male,smoker_no,smoker_yes,region_northeast,region_northwest,region_southeast,region_southwest,log_age,log_bmi,log_charges
0,19,27.900,0,16884.92400,1,0,0,1,0,0,0,1,2.944439,3.328627,9.734176
1,18,33.770,1,1725.55230,0,1,1,0,0,0,1,0,2.890372,3.519573,7.453302
2,28,33.000,3,4449.46200,0,1,1,0,0,0,1,0,3.332205,3.496508,8.400538
3,33,22.705,0,21984.47061,0,1,1,0,0,1,0,0,3.496508,3.122585,9.998092
4,32,28.880,0,3866.85520,0,1,1,0,0,1,0,0,3.465736,3.363149,8.260197
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1333,50,30.970,3,10600.54830,0,1,1,0,0,1,0,0,3.912023,3.433019,9.268661
1334,18,31.920,0,2205.98080,1,0,1,0,1,0,0,0,2.890372,3.463233,7.698927
1335,18,36.850,0,1629.83350,1,0,1,0,0,0,1,0,2.890372,3.606856,7.396233
1336,21,25.800,0,2007.94500,1,0,1,0,0,0,0,1,3.044522,3.250374,7.604867


In [32]:
log_predictors=['age','bmi','children','sex_female','sex_male','smoker_no','smoker_yes','region_northeast','region_northwest','region_southeast','region_southwest']
log_outcome=['log_charges']
log_model=LinearRegression()
log_model.fit(data[log_predictors],data[log_outcome])
print('Intercept',log_model.intercept_)
log_coeficientes=log_model.coef_.tolist()[0]
for name, coef in zip(log_predictors,log_coeficientes):
    print(f'{name}:{coef}')

Intercept [7.68252713]
age:0.034581640531723676
bmi:0.013374822725804442
children:0.10185684855526678
sex_female:0.03770822212164485
sex_male:-0.03770822212164493
smoker_no:-0.7771613929487877
smoker_yes:0.777161392948788
region_northeast:0.08748413289555435
region_northwest:0.023696569257015243
region_southeast:-0.06971261320991577
region_southwest:-0.0414680889426537


In [33]:
model_log=sm.OLS(data[log_outcome],data[log_predictors].assign(const=1))
results_log=model_log.fit()
results_log.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            log_charges   R-squared:                       0.768
Model:                            OLS   Adj. R-squared:                  0.767
Method:                 Least Squares   F-statistic:                     549.8
Date:                Thu, 26 May 2022   Prob (F-statistic):               0.00
Time:                        02:15:24   Log-Likelihood:                -808.52
No. Observations:                1338   AIC:                             1635.
Df Residuals:                    1329   BIC:                             1682.
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
age                  0.0346      0.001     39.655      0.000       0.033       0.036
bmi                  0.0134      0.002      6.381      0.000       0.009       0.017
children             0.1019      0.010     10.085      0.000       0.082       0.122
sex_female           1.7449      0.020     88.435      0.000       1.706       1.784
sex_male             1.6695      0.020     82.844      0.000       1.630       1.709
smoker_no            0.9301      0.021     44.948      0.000       0.889       0.971
smoker_yes           2.4844      0.023    108.119      0.000       2.439       2.529
region_northeast     0.9411      0.022     42.754      0.000       0.898       0.984
region_northwest     0.8773      0.022     39.725      0.000       0.834       0.921
region_southeast     0.7839      0.024     32.338      0.000       0.736       0.831
region_southwest     0.8121      0.023     35.639      0.000       0.767       0.857
const                3.4145      0.032    108.219      0.000       3.353       3.476
==============================================================================
Omnibus:                      463.882   Durbin-Watson:                   2.046
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1673.760
Skew:                           1.679   Prob(JB):                         0.00
Kurtosis:                       7.330   Cond. No.                     7.14e+17
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 6.91e-30. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

In [34]:
log_predictors2=['age','children','sex_male','smoker_no','region_northeast']
log_outcome2=['log_charges']
log_model3=sm.OLS(data[log_outcome2],data[log_predictors2].assign(const=1))
resultslog2=log_model3.fit()
resultslog2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            log_charges   R-squared:                       0.760
Model:                            OLS   Adj. R-squared:                  0.760
Method:                 Least Squares   F-statistic:                     845.8
Date:                Thu, 26 May 2022   Prob (F-statistic):               0.00
Time:                        02:15:24   Log-Likelihood:                -829.70
No. Observations:                1338   AIC:                             1671.
Df Residuals:                    1332   BIC:                             1703.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
age                  0.0352      0.001     40.082      0.000       0.034       0.037
children             0.1029      0.010     10.044      0.000       0.083       0.123
sex_male            -0.0684      0.025     -2.766      0.006      -0.117      -0.020
smoker_no           -1.5504      0.031    -50.595      0.000      -1.610      -1.490
region_northeast     0.0920      0.029      3.195      0.001       0.035       0.148
const                8.8500      0.047    186.332      0.000       8.757       8.943
==============================================================================
Omnibus:                      447.224   Durbin-Watson:                   2.051
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1536.127
Skew:                           1.634   Prob(JB):                         0.00
Kurtosis:                       7.108   Cond. No.                         175.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Cross-Validation

In [35]:
#Particiones.
k=35

In [ ]:
slices=np.array_split(data,k)
slices

In [37]:
slices[0]


,age,bmi,children,charges,sex_female,sex_male,smoker_no,smoker_yes,region_northeast,region_northwest,region_southeast,region_southwest,log_age,log_bmi,log_charges
0,19,27.900,0,16884.92400,1,0,0,1,0,0,0,1,2.944439,3.328627,9.734176
1,18,33.770,1,1725.55230,0,1,1,0,0,0,1,0,2.890372,3.519573,7.453302
2,28,33.000,3,4449.46200,0,1,1,0,0,0,1,0,3.332205,3.496508,8.400538
3,33,22.705,0,21984.47061,0,1,1,0,0,1,0,0,3.496508,3.122585,9.998092
4,32,28.880,0,3866.85520,0,1,1,0,0,1,0,0,3.465736,3.363149,8.260197
5,31,25.740,0,3756.62160,1,0,1,0,0,0,1,0,3.433987,3.248046,8.231275
6,46,33.440,1,8240.58960,1,0,1,0,0,0,1,0,3.828641,3.509753,9.016827
7,37,27.740,3,7281.50560,1,0,1,0,0,1,0,0,3.610918,3.322875,8.893093
8,37,29.830,2,6406.41070,0,1,1,0,1,0,0,0,3.610918,3.395515,8.765054
9,60,25.840,0,28923.13692,1,0,1,0,0,1,0,0,4.094345,3.251924,10.272397


/usr/local/lib/python3.7/dist-packages/pandas/core/internals/construction.py:540: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  values = np.array([convert(v) for v in values])


,0,age,bmi,charges,children,log_age,log_bmi,log_charges,region_northeast,region_northwest,region_southeast,region_southwest,sex_female,sex_male,smoker_no,smoker_yes
39,NaN,60.0,39.90,48173.3610,0.0,4.094345,3.686376,10.782561,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0
40,NaN,24.0,26.60,3046.0620,0.0,3.178054,3.280911,8.021605,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
41,NaN,31.0,36.63,4949.7587,2.0,3.433987,3.600868,8.507094,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
42,NaN,41.0,21.78,6272.4772,1.0,3.713572,3.080992,8.743927,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0
43,NaN,37.0,30.80,6313.7590,2.0,3.610918,3.427515,8.750486,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27,age bmi children charges sex_...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28,age bmi children charges sex_...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29,age bmi children charges sex_...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30,age bmi children charges sex_...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [38]:
rsquared_results=[]
f_statistic_results=[]
p_values_results=[]
for i in range(len(slices)):
    reserved_data=slices[i]
    left_side=pd.DataFrame(slices[:i])
    right_side=pd.DataFrame(slices[i+1:])
    remaining_data=pd.concat([left_side,right_side])
    outcome=['log_charges']
    predictors=['age','children','sex_male','smoker_no','region_northeast']
    #Entrenamiento
    model=LinearRegression()
    model.fit(remaining_data[predictors],remaining_data[outcome])
    rsquared=model.score(reserved_data[predictors],reserved_data[outcome])
    f_statistic,p_values=f_regression(reserved_data[predictors],reserved_data[outcome])
    rsquared_results.append(rsquared)
    f_statistic_results.append(f_statistic)
    p_values_results.append(p_values)
mean_R2=sum(rsquared_results)/len(rsquared_results)
mean_f_statistic_result=sum(f_statistic_results)/len(f_statistic_results)
mean_p_values_result=sum(p_values_results)/len(p_values_results)
print(mean_R2,mean_f_statistic_result)

/usr/local/lib/python3.7/dist-packages/pandas/core/internals/construction.py:540: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  values = np.array([convert(v) for v in values])


KeyError: ignored